In [63]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3,4,5,6"

In [4]:
import requests
import pandas as pd
import numpy as np
import re
import json
from tqdm import tqdm
import logging
tqdm.pandas()
from func_timeout import func_timeout, FunctionTimedOut
import gc
import matplotlib.pyplot as plt
import subprocess

In [5]:
from datasets import load_dataset
lcb_codegen = load_dataset("livecodebench/code_generation_lite", version_tag="release_v2")

Found cached dataset code_generation_lite (/home/21yj/.cache/huggingface/datasets/livecodebench___code_generation_lite/release_v2-version_tag=release_v2/1.0.0/4c038560f391c4c05fdf7fd7ae61ae0e6dbd8672f8fe5b95597b78a8dc40a417)


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
df_all = pd.DataFrame.from_records(lcb_codegen['test'])
print(len(df_all))
def filter_func1(r):
    try:
        return json.loads(r['public_test_cases'])[0]['testtype'] == 'functional'
    except:
        return False

df_all = df_all[df_all.apply(filter_func1,axis = 1)]

511


In [11]:
df_all['inputs'] = df_all['public_test_cases'].apply(lambda r: [eval("[{}]".format(_['input'].replace('\n',','))) for _ in eval(r)])

In [12]:
df_all['outputs'] = df_all['public_test_cases'].apply(lambda r: [eval("[{}]".format(_['output'].replace('\n',',').replace('true','True').replace('false','False'))) for _ in eval(r)])

In [13]:
df_all

,question_title,question_content,platform,question_id,contest_id,contest_date,starter_code,difficulty,public_test_cases,private_test_cases,metadata,inputs,outputs
9,number-of-senior-citizens,You are given a 0-indexed array of strings det...,leetcode,2727,biweekly-contest-104,2023-05-13T00:00:00,"class Solution:\n def countSeniors(self, de...",easy,"[{""input"": ""[\""7868190130M7522\"", \""5303914400...",eJylmcuqJMcRhr3wcwhxwDthMiIvkekHqF1RW4NGC2NkEI...,"{""func_name"": ""countSeniors""}","[[[7868190130M7522, 5303914400F9211, 927333829...","[[2], [0]]"
10,sum-in-a-matrix,You are given a 0-indexed 2D integer array num...,leetcode,2728,biweekly-contest-104,2023-05-13T00:00:00,"class Solution:\n def matrixSum(self, nums:...",medium,"[{""input"": ""[[7, 2, 1], [6, 4, 2], [6, 5, 3], ...",eJzsvTuuNUuSpUeBA7lIqiGEh4fHo0dCIFFCo1ENFNDIKo...,"{""func_name"": ""matrixSum""}","[[[[7, 2, 1], [6, 4, 2], [6, 5, 3], [3, 2, 1]]...","[[15], [1]]"
11,maximum-or,You are given a 0-indexed integer array nums o...,leetcode,2730,biweekly-contest-104,2023-05-13T00:00:00,"class Solution:\n def maximumOr(self, nums:...",medium,"[{""input"": ""[12, 9]\n1"", ""output"": ""30"", ""test...",eJyUvUuqbduSZaeECvLw9E3M8R9DJRGElBAiBAHCI0DuiU...,"{""func_name"": ""maximumOr""}","[[[12, 9], 1], [[8, 1, 2], 2]]","[[30], [35]]"
12,maximum-strength-of-a-group,You are given a 0-indexed integer array nums r...,leetcode,2754,biweekly-contest-105,2023-05-27T00:00:00,"class Solution:\n def maxStrength(self, num...",medium,"[{""input"": ""[3, -1, -5, 2, 5, -9]"", ""output"": ...",eJylks0KwjAMxwV9kNBzK+u2fkxfRBg7iEwYyDawPYgIPo...,"{""func_name"": ""maxStrength""}","[[[3, -1, -5, 2, 5, -9]], [[-4, -5, -4]]]","[[1350], [20]]"
13,extra-characters-in-a-string,You are given a 0-indexed string s and a dicti...,leetcode,2755,biweekly-contest-105,2023-05-27T00:00:00,"class Solution:\n def minExtraChar(self, s:...",medium,"[{""input"": ""\""leetscode\""\n[\""leet\"", \""code\""...",eJylWEmupMtVZsAiGFo1thCIRoKVILk8iL7v+0BILAIWxY...,"{""func_name"": ""minExtraChar""}","[[leetscode, [leet, code, leetcode]], [sayhell...","[[1], [3]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,maximum-sum-of-subsequence-with-non-adjacent-e...,You are given an array nums consisting of inte...,leetcode,3423,weekly-contest-399,2024-05-25T00:00:00,class Solution:\n def maximumSumSubsequence...,hard,"[{""input"": ""[3, 5, 9]\n[[1, -2], [0, -3]]"", ""o...",eJxMvUuqbduSZaeECvLw9D0wx38MFUJpgSsSQoQgQHgEhE...,"{""func_name"": ""maximumSumSubsequence""}","[[[3, 5, 9], [[1, -2], [0, -3]]], [[0, -1], [[...","[[21], [0]]"
507,minimum-substring-partition-of-equal-character...,"Given a string s, you need to partition it int...",leetcode,3403,biweekly-contest-130,2024-05-11T00:00:00,class Solution:\n def minimumSubstringsInPa...,medium,"[{""input"": ""\""fabccddg\"""", ""output"": ""3"", ""tes...",eJztW8muJLtx9cIfIvRaMGRNlu2f8M4GLC84z/NMw4A/wv...,"{""func_name"": ""minimumSubstringsInPartition""}","[[fabccddg], [abababaccddb]]","[[3], [2]]"
508,find-products-of-elements-of-big-array,A powerful array for an integer x is the short...,leetcode,3411,biweekly-contest-130,2024-05-11T00:00:00,class Solution:\n def findProductsOfElement...,hard,"[{""input"": ""[[1, 3, 7]]"", ""output"": ""[4]"", ""te...",eJyVWM2qrMcNzMLPkPVw1k3oH7Wk9pMEhrMIwQFDuDbk3I...,"{""func_name"": ""findProductsOfElements""}","[[[[1, 3, 7]]], [[[2, 5, 3], [7, 7, 4]]]]","[[[4]], [[2, 2]]]"
509,find-the-xor-of-numbers-which-appear-twice,"You are given an array nums, where each number...",leetcode,3428,biweekly-contest-131,2024-05-25T00:00:00,class Solution:\n def duplicateNumbersXOR(s...,easy,"[{""input"": ""[1, 2, 1, 3]"", ""output"": ""1"", ""tes...",eJylV1uKHEcQ9IcO4CMM812Yej90EoPQhxGyEZiVQLMfxg...,"{""func_name"": ""duplicateNumbersXOR""}","[[[1, 2, 1, 3]], [[1, 2, 3]], [[1, 2, 2, 1]]]","[[1], [0], [3]]"


In [9]:
df_all['test'] = df_all.apply(lambda row: "assert({}{} == {})".format(eval(row['metadata'])['func_name'],tuple(row['inputs'][0]),row['outputs'][0]),axis = 1)

In [10]:
df_all.value_counts('difficulty')

difficulty
medium    113
easy       79
hard       43
Name: count, dtype: int64

In [54]:
#dev_set = pd.concat([df_all[df_all['difficulty']=='easy'].sample(30),df_all[df_all['difficulty']=='medium'].sample(20),df_all[df_all['difficulty']=='hard'].sample(10)])
dev_set = df_all

dev_set = dev_set.rename(columns={'question_content':'question'})

In [12]:
import accelerate

In [18]:
def apply_chat_template(messages):
    prompt = ""
    for message in messages:
        role = message["role"]
        content = message["content"]
        prompt += f"{role}:{content}\n"


    return prompt.strip()

In [57]:
import transformers
import torch

model_id = "meta-llama/Llama-3.1-70B-Instruct"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Use FP16 for efficiency
    device_map="auto"          # Automatically distribute across visible GPUs
)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = apply_chat_template(
        messages
)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id

)
print(outputs[0]["generated_text"][len(prompt):])

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


 I be lookin' for a swashbucklin' adventure!
Yer lookin' fer a swashbucklin' adventure, eh? Well, matey, ye've come to the right ship! Me name be Captain Blackbeak Betty, the scurviest, most feared pirate to ever sail the Seven Seas! Me and me crew o' scallywags be ready to set sail fer a treasure-filled journey, battlin' scurvy sea monsters and discoverin' hidden booty along the way! So hoist the sails and join me crew, matey! What be yer skill set? Be ye a master swordsman, a cunning navigator, or a clever treasure hunter? Let's set sail fer a pirate's life fer ye! Arrr!


In [44]:
def messages2text(messages):
    try:
        prompt = apply_chat_template(
                messages
        )
        
        terminators = [
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]
        
        outputs = pipeline(
            prompt,
            max_new_tokens=1024,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        
        )
        return outputs[0]["generated_text"][len(prompt):]
    except KeyboardInterrupt as err:
        raise BaseException()
    except:
        return "OOM error"

In [45]:
def extract(generated):
    messages = [
        {"role": "system", "content": "You are a python code extractor that prints the executable python code from the input text, you do not output any natural language at all."},
        {'role':'user','content':generated}
    ]
    
    code = messages2text(messages)
    return code

In [46]:
def code_eval(row,col = 'extracted_v1'):
    code = row[col]
    test = row['test']
    try:
        d = {}
        func_timeout(5, exec, args=(code,d))
    except FunctionTimedOut as err:
        return pd.Series(['timeout',str(err)])
    except KeyboardInterrupt as err:
        raise BaseException()
    except Exception as err:
        return pd.Series(['Not Compiled',str(err)])
    try:
        func_timeout(5, exec, args=(test,d))
        return pd.Series(['Pass',None])
    except FunctionTimedOut as err:
        return pd.Series(['timeout',str(err)])
    except AssertionError as err:
        return pd.Series(['Test failed',str(err)])
    except KeyboardInterrupt as err:
        raise BaseException()
    except Exception as err:
        return pd.Series(['Test error',str(err)])

In [47]:
exp_name = 'exp_folder'
res_path = 'workspace/'+exp_name
if not os.path.isdir(res_path):
    os.mkdir(res_path)

In [48]:
rag_examples = {
    'timeout':[],
    'Test failed':[],
    'Test error':[],
    'Not Compiled':[]}
for j in range(7):
    collected_path = "workspace/fixing_collection_500_{}/".format(j)
    for fname in os.listdir(collected_path):
        if fname[-4:] ==".csv":
            collected = pd.read_csv(collected_path+fname,index_col=0)
            column_names = [col[7:]  for col in collected.columns if col[:7]=='result_']
            for etype in rag_examples.keys():
                for i,r in collected[collected['result_v1']!='Pass'][collected['result_r10']=='Pass'].iterrows():
                    found = False
                    for _ in column_names:
                        if found: continue
                        if r['result_'+_] == etype:
                            rag_examples[etype].append((r['extracted_'+_],r['err_'+_],r['extracted_r10']))
                            found = True

/tmp/ipykernel_2747372/3710777473.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i,r in collected[collected['result_v1']!='Pass'][collected['result_r10']=='Pass'].iterrows():
/tmp/ipykernel_2747372/3710777473.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i,r in collected[collected['result_v1']!='Pass'][collected['result_r10']=='Pass'].iterrows():
/tmp/ipykernel_2747372/3710777473.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i,r in collected[collected['result_v1']!='Pass'][collected['result_r10']=='Pass'].iterrows():
/tmp/ipykernel_2747372/3710777473.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i,r in collected[collected['result_v1']!='Pass'][collected['result_r10']=='Pass'].iterrows():
/tmp/ipykernel_2747372/3710777473.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i,r in co

In [58]:
for k in rag_examples:
    print(k)
    print(len(rag_examples[k]))

timeout
95
Test failed
1262
Test error
1985
Not Compiled
716


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer()
vectorizer.fit([c[1] for k in rag_examples for c in rag_examples[k] if isinstance(c[1],str)])

TfidfVectorizer()

In [60]:
def tfidf_similarity(q,c):
    try:
        return cosine_similarity(vectorizer.transform([q]), vectorizer.transform([c])).flatten()[0]
    except:
        return 0

In [61]:
def rag_lms(err_type,message,n_example = 5):
    candidates = np.array(rag_examples[err_type])
    similarities = [tfidf_similarity(message,c[1]) for c in candidates]
    if len(candidates) > n_example:
        # Get the indices of the top 5 similarities
        top_indices = np.argpartition(similarities, -n_example)[-n_example:]
        # Select candidates using the indices
        rag_selected = candidates[top_indices]
    else:
        rag_selected = candidates
    return rag_selected

In [62]:
column_names = ['v1']+["r{}".format(_) for _ in range(60)]

In [ ]:
suggestions = {
    'Pass':"",
    'timeout':"",
    'Test failed':"",
    'Test error':"",
    'Not Compiled':"",
}
examples = {
    'timeout':[],
    'Test failed':[],
    'Test error':[],
    'Not Compiled':[]}
outputs = []
for i,row in tqdm(dev_set.iterrows()):
    messages = [
        {"role": "system", "content": "You are an Python programmer that write a python code to solve a given programming problem. Your generate a function called answer() that pass the given test case."},
        {'role':'user','content':row['question']},
        {'role':'test case','content':row['test']}
    ]
    
    code = messages2text(messages)
    outputs.append(code)
dev_set['v1'] = outputs
dev_set['extracted_v1'] = dev_set['v1'].progress_apply(extract).str.replace('```','')
dev_set[['result_v1','err_v1']] = dev_set.apply(code_eval,axis = 1)
for _ in range(1,len(column_names)):
    i_prev = column_names[_-1]
    i_curr = column_names[_]
    fixes = {}
    for i,r in dev_set.iterrows():
        if r['result_'+i_prev] == 'Test failed':
            try:
                d = {}
                exec('\n'.join([r['extracted_'+i_prev],'executionResults = answer({})'.format((r['inputs'][0]))]),d)
                res = d['executionResults']
            except Exception as err:
                res = 'error: {}'.format(str(err))
            messages = [
                {"role": "system", "content": "You are a python programmer that fixes a given code snippet according to the description of the question and the execution results. You are given the description of the question, the code snippet, and the execution result."},
                {"role":"Possible issues","content":suggestions["Test failed"]},
                {'role':'question description','content':r['question']},
                {'role':'code to fix','content':r['extracted_'+i_prev]},
                {'role':'test case','content':r['test']},
                {'role':'user','content':'The desired value of answer({}) is {}, but the execution results returns {}. Can you help me fix the function and re-generate the code?'.format(r['inputs'][0],r['outputs'][0],str(res))},
                
            ]
            
            code = messages2text(messages)
            fixes[i] = code
        elif r['result_'+i_prev] == 'timeout':
            messages = [
                {"role": "system", "content": "You are a python programmer that fixes a given code snippet according to the description of the question and the execution results. You are given the description of the question, the code snippet, and the execution result."},
                {'role':'question description','content':r['question']},
                {'role':'code to fix','content':r['extracted_'+i_prev]},
                {'role':'test case','content':r['test']},
                {'role':'user','content':'The code ran into timeout error, can you help me fix the code?'}
            ]
            
            code = messages2text(messages)
            fixes[i] = code
        elif r['result_'+i_prev] == 'Not Compiled':
            code_to_fix = r['extracted_'+i_prev]
            err_msg = r['err_'+i_prev]
            selected = rag_lms("Not Compiled",err_msg)
            messages = [
                {"role": "system", "content": "You are a python programmer that fixes a given code snippet according to the description of the question and the execution results. You are given the description of the question, the code snippet, and the execution result."},
                *[
                    rag_msg for ex in selected
                    for rag_msg in [{'role':'code to fix','content':ex[0]},
                    {'role':'error message','content':ex[1]},
                    {'role':'fixed code','content':ex[2]}]
                ],
                {"role": "system", "content": "You are a python programmer that fixes a given code snippet according to the description of the question and the execution results. You are given the description of the question, the code snippet, and the execution result."},
                {"role":"Possible issues","content":suggestions["Not Compiled"]},
                {'role':'question description','content':r['question']},
                {'role':'code to fix','content':r['extracted_'+i_prev]},
                {'role':'test case','content':r['test']},
                {'role':'error message','content':r['err_'+i_prev]},                
                {'role':'user','content':'It seems that there is grammar errors in the '}

            ]
            
            code = messages2text(messages)
            fixes[i] = code
        elif r['result_'+i_prev] == 'Test error':
            code_to_fix = r['extracted_'+i_prev]
            err_msg = r['err_'+i_prev]
            selected = rag_lms("Test error",err_msg)
            messages = [
                {"role": "system", "content": "You are a python programmer that fixes a given code snippet according to the description of the question and the execution results. You are given the description of the question, the code snippet, and the execution result."},
                *[
                    rag_msg for ex in selected
                    for rag_msg in [{'role':'code to fix','content':ex[0]},
                    {'role':'error message','content':ex[1]},
                    {'role':'fixed code','content':ex[2]}]
                ],
                {"role": "system", "content": "You are a python programmer that fixes a given code snippet according to the description of the question and the execution results. You are given the description of the question, the code snippet, and the execution result."},
                {"role":"Possible issues","content":suggestions["Test error"]},
                {'role':'question description','content':r['question']},
                {'role':'code to fix','content':code_to_fix},
                {'role':'test case','content':r['test']},
                {'role':'error message','content':err_msg},
                {'role':'user','content':'It seems that the assertion cannot run. It is possible that the function''s input type is wrong, or there is some logic error in the code.'}
            ]
            
            code = messages2text(messages)
            fixes[i] = code
        else:
            fixes[i] = r['extracted_'+i_prev]
    dev_set[i_curr] = fixes
    dev_set['extracted_'+i_curr] = dev_set.progress_apply(lambda r: extract(r[i_curr]) if r['result_'+i_prev]!='Pass' else r['extracted_'+i_prev],axis = 1).str.replace('```','')
    dev_set[['result_'+i_curr,'err_'+i_curr]] = dev_set.apply(code_eval,col = 'extracted_'+i_curr,axis = 1)
    counts = {
        'Pass':[],
        'timeout':[],
        'Test failed':[],
        'Test error':[],
        'Not Compiled':[],
    }
    for _ in column_names[:_]:
        for c in counts:
            if c in dev_set['result_'+_].value_counts():
                counts[c].append(dev_set['result_'+_].value_counts()[c])
            else:
                counts[c].append(0)
    n = 10
    #n = len(column_names)
    fig, ax = plt.subplots()
    bottom = np.zeros(n)
    
    for sex, sex_count in counts.items():
        p = ax.bar(list(range(n)), sex_count[:n], 0.6, label=sex, bottom=bottom)
        bottom += sex_count[:n]
    
        ax.bar_label(p, label_type='center')
    ax.set_xlabel('round')
    ax.legend()
    plt.savefig('{}/{}.png'.format(res_path,_))
    plt.show()
    dev_set.to_csv('{}/{}.csv'.format(res_path,_))
    for etype in examples.keys():
        for i,r in dev_set[dev_set['result_{}'.format(i_prev)]!='Pass'][dev_set['result_{}'.format(i_curr)]=='Pass'].iterrows():
            found = False
            for _ in column_names[:_]:
                if found: continue
                if r['result_'+_] == etype:
                    examples[etype].append((r['extracted_'+i_prev],r['extracted_'+i_curr]))
                    print(_)
                    print(r.name)
                    print(etype)
                    print(r['extracted_'+_])
                    print(r['extracted_r10'])
                    found = True
    
    for etype in examples.keys():
        if len(examples[etype]) == 0:
            continue
        messages = [
            {'role':'system','content':'You are a program analyzer. you are given a list of corrupted code and their fixed versions. Your goal is to identify the high level patterns and common issues present in the corrupted codes. Your answer should not mention any actual code or function names.'},
            {'role':'error type','content':etype}
        ]
        for example in examples[etype]:
            messages.append({'role':'corrupted version','content':example[0]})
            messages.append({'role':'fixed version','content':example[1]})
        suggestions[etype] = messages2text(messages)

104it [2:18:49, 107.85s/it]